In [3]:
!pip install -q jupyter-dash
!pip install -q pyTigerGraph
# !pip install -q dash_daq
# !pip install dash-extensions

from jupyter_dash import JupyterDash
import dash 
import dash_table

import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pyTigerGraph as tg
import pandas as pd


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  # This is added back by InteractiveShellApp.init_path()


In [4]:
TG_HOST = "https://healthcare.i.tgcloud.io" 
TG_USERNAME = "tigergraph"  
TG_PASSWORD = "tigergraph"  
TG_GRAPHNAME = "faers" 

In [5]:
conn = tg.TigerGraphConnection(host=TG_HOST, username=TG_USERNAME, password=TG_PASSWORD, graphname=TG_GRAPHNAME)
conn.apiToken = conn.getToken(conn.createSecret())

print(conn.gsql('''ls''', options=[]))

---- Graph faers
Vertex Types:
- VERTEX ReportedCase(PRIMARY_ID primaryid UINT, caseid UINT, caseversion UINT, fda_dt DATETIME, mfr_sndr STRING, reporter_country STRING, occr_country STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
- VERTEX PharmaCompany(PRIMARY_ID primaryid STRING, mfr_sndr STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
- VERTEX DrugSequence(PRIMARY_ID sequence_id STRING, seqid UINT, role_cod STRING, drugname STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
- VERTEX Drug(PRIMARY_ID drug_id STRING, drugname STRING, prod_ai STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
- VERTEX Reaction(PRIMARY_ID reaction_id STRING, pt STRING, drug_rec_act STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
- VERTEX Outcome(PRIMARY_ID outcome_id STRING, outc_code STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
- VERTEX ReportSource(PRIMARY_ID reportsource_id STRING, rpsr_code STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
- VERTEX Patient(PRIMARY_ID primaryid UINT, age FLOAT, age_code STRING, age_grp STRING, sex STRING, w

In [6]:
app = JupyterDash(__name__,suppress_callback_exceptions=True)

In [7]:
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sidebar = html.Div(
    [
        html.H2("FAERS Data Visualization", className="display-4"),
        html.Hr(),
        html.P(
            "A simple demo to visualize the most reported cases for a company. Please select the company from the dropdown", className="lead"
        ),
        dcc.Dropdown(
          id = "bar_type",
          options=[
            {"label": "PFIZER", "value": "PFIZER"},
            {"label": "GILEAD", "value": "GILEAD"},
            {"label": "INCYTE", "value": "INCYTE"},
            {"label": "NOVARTIS", "value": "NOVARTIS"}

          ],
          value="PFIZER"
      )
    ],
    style=SIDEBAR_STYLE,
    
)

content = html.Div(
    [
     dcc.Graph(id="bar-fig")
    ],
    style=CONTENT_STYLE,
)

app.layout = html.Div([sidebar,content])


In [8]:
@app.callback(
   Output('bar-fig', 'figure'),
   Input('bar_type', 'value'),
)
def update_graph(value):
  com_name = str(value)
  res = conn.runInstalledQuery("mostReportedDrugsForCompany_v2", params={"companyName":str(com_name),"k":5,"role":"PS"})[0]["TopDrugs"]
  df = pd.DataFrame(res)
  split = [df, pd.DataFrame(df['attributes'].tolist()).iloc[:, :3]]
  splitDf = pd.concat(split, axis=1).drop('attributes', axis=1)
  bar = px.bar(splitDf, x="v_id", y="@numCases", barmode='overlay')
  return bar

In [9]:
app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>